In [2]:
import pandas as pd
import sklearn as sk

1. Preparar Data

1.1 Leer Dataset

Los imput (x) seran los comentarios o reviews

Los Outputs (y) seran los sentimientos 

In [3]:
df_review = pd.read_csv('IMDB Dataset.csv')
df_review

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
#Utilizamos una data mas pequeña para desequilibrar bbdd y poder entrenar mas facil el robot
df_positivo = df_review[df_review['sentiment']=='positive'][:9000]
df_negativo = df_review[df_review['sentiment']=='negative'][:1000]
df_review_des = pd.concat([df_positivo, df_negativo])
df_review_des.value_counts('sentiment')

sentiment
positive    9000
negative    1000
Name: count, dtype: int64

1.2 Dataset desbalanceado

In [5]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
#Para balancear esta data
df_review_bal, df_review_bal['sentiment'] = rus.fit_resample(df_review_des[['review']], df_review_des['sentiment'])

In [6]:
#Para chequear que la data este balanceada
df_review_bal.value_counts(['sentiment'])

sentiment
negative     1000
positive     1000
Name: count, dtype: int64

1.3 Separar la data para entrenar (train) y la data para testear(test)

In [7]:
from sklearn.model_selection import train_test_split

#asignar que porcentaje de la bbdd porcentaje para testear y entrenar
train, test= train_test_split(df_review_bal, test_size=0.33, random_state=42)

In [8]:
train_x, train_y = train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']

Transformar la data de texto a data numerica (Bag of Words)
#countVectorizer (frecuencia que una palabra aparece)
#tfidf (relevancia que tiene una palabra dentro de una oracion que no este repetida en otra oracion.)

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


In [10]:
tfidf = TfidfVectorizer (stop_words='english')
train_x_vector = tfidf.fit_transform(train_x)
test_x_vector = tfidf.transform(test_x)


**Support Vector Machines (SVM)**

In [11]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')
svc.fit(train_x_vector, train_y)

SVC(kernel='linear')

***Testeo***

In [12]:
print(svc.predict(tfidf.transform(['A good movie'])))
print(svc.predict(tfidf.transform(['An excellent movie'])))
print(svc.predict(tfidf.transform(['I did not like this movie at all I gave this movie away'])))

['positive']
['positive']
['negative']


**Decision Tree**


In [13]:
from sklearn.tree import DecisionTreeClassifier

dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_x_vector, train_y)

DecisionTreeClassifier()

**Naive Bayes**

In [14]:
 from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(train_x_vector.toarray(), train_y)

GaussianNB()

**Regresion logistica**

In [15]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_x_vector, train_y)

LogisticRegression()

**Evaluacion de los modelos**

1. Score del modelo

In [23]:
svc.score(test_x_vector, test_y)
#este es el mejor modelo de los 4 porque saco mayor puntuacion

0.8303030303030303

In [17]:
dec_tree.score(test_x_vector, test_y)

0.6772727272727272

In [18]:
gnb.score(test_x_vector.toarray(), test_y)

0.6393939393939394

In [19]:
lr.score(test_x_vector, test_y)

0.8151515151515152

2. F1 Score

In [24]:
from sklearn.metrics import f1_score

f1_score(test_y, svc.predict(test_x_vector),
         labels=['positive', 'negative'],
         average=None)

array([0.83768116, 0.82222222])

3. Reporte de clasificacion

In [27]:
from sklearn.metrics import classification_report

print(classification_report(test_y, svc.predict(test_x_vector),
         labels=['positive', 'negative']))

              precision    recall  f1-score   support

    positive       0.81      0.86      0.84       335
    negative       0.85      0.80      0.82       325

    accuracy                           0.83       660
   macro avg       0.83      0.83      0.83       660
weighted avg       0.83      0.83      0.83       660



4. Confusion Matrix

In [29]:
from sklearn.metrics import confusion_matrix

confusion_matrix (test_y, svc.predict(test_x_vector),
         labels=['positive', 'negative'])

array([[289,  46],
       [ 66, 259]])

**5. Optimizacion del modelo**

5.1 GridSearchCV

In [35]:
from sklearn.model_selection import GridSearchCV

parametros = {'C':[1,4,8,16,32], 'kernel':['lineal','rbf']}
#'C' es un parametro de penalizacion, un termino de error.
# Indica a la optimizacion del algoritmo cuanto error es soportable

#'kernel' es parte de nuestro sistema que hace todos los procesamiento.
# Debemos indicar que tipo de funcion queremos utilizar (lineales, polinomicas o RBF)
svc = SVC()
svc_grid = GridSearchCV(svc, parametros, cv=5)
svc_grid.fit(train_x_vector, train_y)

c:\Users\Agus\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Agus\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Agus\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\Agus\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", l

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 4, 8, 16, 32], 'kernel': ['lineal', 'rbf']})

In [40]:
print(svc_grid.best_estimator_)
print(svc_grid.best_params_)
print(svc_grid.best_score_)

#paso de medir 0.8303030303030303 y despues de la otimizacion del modelo 0.8343283582089553

SVC(C=4)
{'C': 4, 'kernel': 'rbf'}
0.8343283582089553
